In [2]:
import memcache
import random
import time
import tkinter as tk
from tkinter import ttk  
from threading import Thread



In [ ]:

# Connect to the Memcached server
mc = memcache.Client(['XXXX'], debug=0)

# Rangos definidos según el puntaje
def get_rank(score):
    if score <= 1000:
        return "Básico"
    elif score <= 3000:
        return "Intermedio"
    elif score <= 5000:
        return "Avanzado"
    else:
        return "Legendario"

# Función para agregar o actualizar un puntaje en el leaderboard
def add_score(name, score):
    leaderboard = mc.get('leaderboard') or {}
    leaderboard[name] = score
    mc.set('leaderboard', leaderboard)

    # Actualizar y guardar historial
    historial_entrada = f"{time.ctime()}: {score}"
    historial = mc.get(f"{name}_historial") or []
    historial.append(historial_entrada)
    mc.set(f"{name}_historial", historial)

# Función para recuperar los 50 mejores puntajes del leaderboard
def get_leaderboard():
    leaderboard = mc.get('leaderboard') or {}
    filtered_leaderboard = {name: score for name, score in leaderboard.items() if name in nicknames}
    sorted_leaderboard = sorted(filtered_leaderboard.items(), key=lambda x: x[1], reverse=True)[:50]
    ranked_leaderboard = [(name, score, get_rank(score)) for name, score in sorted_leaderboard]
    return ranked_leaderboard

# Función para ver el historial de cada jugador
def ver_historial(name):
    historial = mc.get(f"{name}_historial")
    if historial:
        print(f"Historial de {name}:")
        for entrada in historial:
            print(entrada)
    else:
        print(f"No hay historial disponible para {name}.")


# Función para simular actualizaciones en tiempo real
def simulate_live_updates(names, max_updates=10):
    updates = 0
    while updates < max_updates:
        leaderboard = mc.get('leaderboard') or {}
        for name in names:
            current_score = leaderboard.get(name, 0)
            new_score = random.randint(current_score + 1, current_score + 50)
            add_score(name, new_score)

        update_leaderboard_display()  # Actualiza la visualización del leaderboard
        time.sleep(10)  # Esperar 10 segundos antes de la siguiente actualización
        updates += 1

# Función para actualizar la visualización del leaderboard en la interfaz
def update_leaderboard_display():
    for row in tree.get_children():
        tree.delete(row)  # Limpiar el Treeview

    top_players = get_leaderboard()
    for player, score, rank in top_players:
        tree.insert('', 'end', values=(player, score, rank))

# Función para iniciar la simulación en un hilo
def start_simulation():
    simulate_live_updates(nicknames, max_updates=5)

# Crear la ventana principal de Tkinter
root = tk.Tk()
root.title("Leaderboard")

# Etiqueta de título
title_label = tk.Label(root, text="Leaderboard", font=("Arial", 16))
title_label.pack(pady=10)

# Crear el Treeview para mostrar el leaderboard
columns = ('Nickname', 'Score', 'Rank')
tree = ttk.Treeview(root, columns=columns, show='headings')
tree.heading('Nickname', text='Nickname')
tree.heading('Score', text='Score')
tree.heading('Rank', text='Rank')
tree.pack(padx=10, pady=10)

# Botón para iniciar la simulación
start_button = tk.Button(root, text="Iniciar Simulación", command=lambda: Thread(target=start_simulation).start())
start_button.pack(pady=20)

# Lista de nicknames personalizada (50 jugadores)
nicknames = ['DragonSlayer', 'ShadowHunter', 'StarGazer', 'MysticWarrior', 'SpeedDemon',
             'ThunderStrike', 'IronFist', 'NightRider', 'FireMage', 'StormBringer',
             'IceQueen', 'DarkKnight', 'WindWalker', 'SkyRuler', 'BladeMaster',
             'StoneGolem', 'SolarFlare', 'Moonlight', 'CrimsonBlade', 'SilverFox',
             'GoldenLion', 'Phantom', 'SilentAssassin', 'GhostRider', 'Inferno',
             'OceanWave', 'BlackPanther', 'FalconEye', 'Rogue', 'Viper', 
             'WolfSpirit', 'WhiteTiger', 'SteelHawk', 'CobraKing', 'Titan', 
             'Hawkeye', 'StormShadow', 'TigerClaw', 'Lightning', 'Venom',
             'Firestorm', 'CrystalPhoenix', 'Warlock', 'Shaman', 'Necromancer', 
             'Berserker', 'Samurai', 'Valkyrie', 'Ranger', 'Paladin']

# Inicializa con puntajes aleatorios
for nickname in nicknames:
    add_score(nickname, random.randint(100, 6000))

# Ejecutar el bucle principal
root.mainloop()

In [4]:
def buscar_jugador(jugador_nombre):
    leaderboard = mc.get('leaderboard') or {}
    
    if jugador_nombre in leaderboard:
        sorted_leaderboard = sorted(leaderboard.items(), key=lambda x: x[1], reverse=True)
        posicion_jugador = next((i for i, v in enumerate(sorted_leaderboard, 1) if v[0] == jugador_nombre), None)
        puntaje_jugador = leaderboard[jugador_nombre]
        print(f"{jugador_nombre} está en la posición {posicion_jugador} con {puntaje_jugador} puntos")
    else:
        print(f"{jugador_nombre} no se encuentra en el ranking")

def buscar_jugador_por_posicion(posicion):
    leaderboard = mc.get('leaderboard') or {}
    sorted_leaderboard = sorted(leaderboard.items(), key=lambda x: x[1], reverse=True)
    
    if posicion > 0 and posicion <= len(sorted_leaderboard):
        jugador, puntaje = sorted_leaderboard[posicion - 1]
        print(f"El jugador en la posición {posicion} es {jugador} con {puntaje} puntos")
    else:
        print(f"No hay jugador en la posición {posicion}")

# Función para obtener el resumen de jugadores
def resumen_jugadores():
    jugadores_puntajes = []
    leaderboard = mc.get('leaderboard') or {}
    for jugador, puntaje in leaderboard.items():
        puntaje = int(puntaje) if puntaje is not None else 0
        jugadores_puntajes.append((jugador, puntaje))
    
    total_jugadores = len(jugadores_puntajes)
    
    if total_jugadores > 0:
        puntaje_total = sum(puntaje for jugador, puntaje in jugadores_puntajes)
        puntaje_promedio = puntaje_total / total_jugadores
    else:
        puntaje_promedio = 0
    
    print(f"Total de jugadores: {total_jugadores}")
    print(f"Puntaje promedio: {puntaje_promedio:.2f} puntos")

def top_n_jugadores(n):
    leaderboard = mc.get('leaderboard') or {}
    sorted_leaderboard = sorted(leaderboard.items(), key=lambda x: x[1], reverse=True)
    
    top_n = sorted_leaderboard[:n]  # Obtener los primeros n jugadores
    
    if top_n:
        print(f"Top {n} jugadores:")
        for i, (jugador, puntaje) in enumerate(top_n, 1):
            print(f"{i}. {jugador} - {puntaje} puntos")
    else:
        print(f"No hay suficientes jugadores para mostrar el top {n}")

In [ ]:
# Función para actualizar el puntaje de un solo jugador y medir el tiempo
def add_score_con_tiempo(name, score):
    start_time = time.time()  # Inicia el temporizador

    # Obtener el leaderboard actual
    leaderboard = mc.get('leaderboard') or {}
    leaderboard[name] = score
    mc.set('leaderboard', leaderboard)  # Actualizar el leaderboard

    # Actualizar y guardar historial
    historial_entrada = f"{time.ctime()}: {score}"
    historial = mc.get(f"{name}_historial") or []
    historial.append(historial_entrada)
    mc.set(f"{name}_historial", historial)

    end_time = time.time()  # Finaliza el temporizador
    duration = end_time - start_time  # Calcula la duración
    print(f"Tiempo de inserción para {name}: {duration} segundos")

# Ejemplo: mide la inserción para un jugador
jugador = "DragonSlayer"
puntaje = random.randint(100, 6000)
add_score_con_tiempo(jugador, puntaje)

In [ ]:
#ejemplo de uso
buscar_jugador('DragonSlayer')
buscar_jugador_por_posicion(1)  # Ver el jugador en la posición 1
resumen_jugadores() #cantidad de jugadores y el puntaje promedio
top_n_jugadores(10)
